## Name : Swathi Sukumar

```
# This is formatted as code
```

# Integer Programming: Fantasy Football


In this homework, you will use integer programming to select an entry for a fantasy football competition. In these competitions, each participant selects players to compose a fantasy team, and the goal is to choose a set that maximizes the number of points these players will score in real-life games.

Each fantasy entry consists of 9 players. The selection of players is subject to two main types of constraints. The first type restricts the choice of players by position. Namely, a feasible entry contains the following number of players per position (you will find this information for each player in the column 'Position'): 


*   2 FB player
*   2 RB player
*   3 WR player
*   2 TE player


Each player has a fantasy salary (you will find this information for each player in the column 'Salary'), and there is a budget restriction of \$50,000 over the entry. In other words, the sum of the salaries of the players selected for your entry cannot be larger than \$50,000.

Your goal is to identify a feasible entry that maximizes the expected number of points (you will find this information for each player in the column 'Points'). The extraction and basic filtering of the data have been provided to you (so please don't change that!!!). As output, **you should print the number of fantasy points scored by your entry and a dataframe containing only the entries for the 9 players you selected.**



**Curiosity**:  Prof. David Bergman, from the OPIM Department, made a lot of money playing this game: 
https://today.uconn.edu/2021/01/business-professor-wins-2-5-million-fantasy-football-jackpot-using-concepts-teaches-students/


**Data source (04/03/2022)**: https://dailyroto.com/nfl-historical-production-fantasy-points-draftkings-fanduel/





**Rubric:**

  *   Data extraction and preparation (5 pts)
  *   Mathematical Programming formulation (15 pts)
  *   Correct code (60 pts)
  *   Print output (20 pts)


* Comments should be provided for all arguments. For this activity, your code should have **at least one original comment for every three lines of code** (**do not copy and paste comments from your colleagues!!!**). You will lose 25\% of the the points if the comment requirement is not met.
* **You should submit a notebook that can be downloaded and executed without adjustments (i.e., your notebook should work).** If your notebook does not satisfy these requirements, the grade of your assignment will be capped at 50 points.
* **Hunter's guidelines to properly check your file for running standalone without your cached history of variables**: In the top right corner see "RAM" and "Disk" with a dropdown arrow next to it. Click the arrow, then manage sessions. Terminate all sessions, you may get a warning, still choose to terminate. This disconnects from the notebook and flushes out the cache. Now click reconnect and then under runtime, "Run All". This should now either function or get stuck. If stuck, fix and repeat. If not, you're good! **Do that before submitting your notebook on HuskyCT!**



## Setup Your Environment/Imports

In [1]:
# before you do anything...
# mount your drive!
# click folder on the left...
# import modules

%matplotlib inline
from pylab import *

# these commands will help copy the data to the colaboratory

# the following command will try to install pyomo environment

import shutil
import sys
import os.path
import pandas as pd
import numpy as np
from pylab import * # simpler interface to matplotlib

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

if not (shutil.which("cbc") or os.path.isfile("cbc")):
    if "google.colab" in sys.modules:
        !apt-get install -y -qq coinor-cbc
    else:
        try:
            !conda install -c conda-forge coincbc 
        except:
            pass

assert(shutil.which("cbc") or os.path.isfile("cbc"))

from pyomo.environ import *

     |████████████████████████████████| 9.6 MB 3.9 MB/s 
     |████████████████████████████████| 49 kB 2.4 MB/s 
Selecting previously unselected package coinor-libcoinutils3v5.
(Reading database ... 155501 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.10.14+repack1-1_amd64.deb ...
Unpacking coinor-libcoinutils3v5 (2.10.14+repack1-1) ...
Selecting previously unselected package coinor-libosi1v5.
Preparing to unpack .../1-coinor-libosi1v5_0.107.9+repack1-1_amd64.deb ...
Unpacking coinor-libosi1v5 (0.107.9+repack1-1) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.16.11+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.16.11+repack1-1) ...
Selecting previously unselected package coinor-libcgl1.
Preparing to unpack .../3-coinor-libcgl1_0.59.10+repack1-1_amd64.deb ...
Unpacking coinor-libcgl1 (0.59.10+repack1-1) ...
Selecting previously unselected package coinor-libcbc3.
Preparing to unp

## Finding the data we need

In [2]:
# Step 1: copy & paste the shared link you will get from Google Drive
URL = 'https://drive.google.com/file/d/11UI8PheL0cbhqy8rpva2QgA4t_OtZbW7/view?usp=sharing'

# Step 2: copy & paste the command below (this part of the command is fixed)
fixed_path = 'https://drive.google.com/uc?export=download&id='

# Step 3: Extract the file ID from the URL (this part of the command is fixed)
file_path = URL.split('/')[-2]

# Retrieve the CSV data and build a dataframe
df = pd.read_csv(fixed_path + file_path)

# Basic data cleansing (PLEASE DO NOT CHANGE THIS!!!)
del df['DK Points']
del df['DKSal']
df = df[df["Week"] == 10].reset_index(drop=True)
df.rename(columns = {'P': 'Position', 'FDSal': 'Salary', 'FD Points': 'Points'}, inplace = True)

# First lines of the dataframe
df.head()

,Position,Player,Week,Team,Opp,Opp Rank,Opp Position Rank,Salary,Points
0,WR,Deebo Samuel,10,SF,LAR,8,7700,7400,30.3
1,WR,Stefon Diggs,10,BUF,NYJ,8,7900,7500,33.2
2,RB,AJ Dillon,10,GB,SEA,29,5300,4800,26.8
3,WR,CeeDee Lamb,10,DAL,ATL,20,7700,7000,28.6
4,RB,Rhamondre Stevenson,10,NE,CLE,10,5400,4500,30.4


Let $X_i$ be a binary variable that represent if the player is present or not in a particular position

* $X_i \in \{0,1\}, i \in [385]$  

The objective function is 

$Max \sum\limits_{i = 1}^{385}(X_i)*Points$ `(objective function)`

Constraints are

$\sum\limits_{i = 1}^{385}(X_i) =$9

$\sum\limits_{i = 1}^{385}(X_i)*Salary$ <=$50000$


Constraints on position are 

* $\sum\limits_{}X_i$ ==$2$  where $i \in \mathbb{FB}$ (`FB Position`)
* $\sum\limits_{}X_i$ ==$2$  where $i \in \mathbb{RB}$ (`RB Position`)
* $\sum\limits_{}X_i$ ==$3$  where $i \in \mathbb{WR}$ (`WR Position`)
* $\sum\limits_{}X_i$ ==$2$  where $i \in \mathbb{TE}$ (`TE Position`)






In [3]:
# let us create subsets of dataframe for each of the position since we have a constraint for each of the position
# let us create subsets of dataframe for each of the position since we have a constraint for each of the position
# The problem clearly says that we need palyers from eaach of these positions
df_fb=df[df["Position"]=="FB"]
df_rb=df[df["Position"]=="RB"]
df_wr=df[df["Position"]=="WR"]
df_te=df[df["Position"]=="TE"]

In [4]:
df_rb

,Position,Player,Week,Team,Opp,Opp Rank,Opp Position Rank,Salary,Points
2,RB,AJ Dillon,10,GB,SEA,29,5300,4800,26.8
4,RB,Rhamondre Stevenson,10,NE,CLE,10,5400,4500,30.4
5,RB,Darrel Williams,10,KC,LV,22,6000,5400,32.4
8,RB,Jonathan Taylor,10,IND,JAX,16,9400,8100,27.6
9,RB,Christian McCaffrey,10,CAR,ARI,4,9000,8400,26.1
...,...,...,...,...,...,...,...,...,...
369,RB,Peyton Barber,10,LV,KC,17,5000,4200,0.0
374,RB,Buddy Howell,10,LAR,SF,21,4600,4000,0.0
378,RB,Trenton Cannon,10,SF,LAR,16,4500,4000,0.0
381,RB,JaMycal Hasty,10,SF,LAR,16,5000,4000,0.0


In [5]:
#check if the subset has happened correctly
players = range(1,386)
# we need to maximise the points and thats our objective. Hence we store points from dataframe into points variable 
points= df["Points"]
# we cannot exceed the salary by a 50k hence we store salary from dataframe into salary variable so that it can be used in our constraint
salary = df["Salary"]

In [6]:
# declare the model
model = ConcreteModel()
# There are 385 players in total and hence we need to create an array of 1-385 players, the binary variable will indicate if each of the player are selected or not
players = range(1,386)
# we need to maximise the points and thats our objective. Hence we store points from dataframe into points variable 
points= df["Points"]
# we cannot exceed the salary by a 50k hence we store salary from dataframe into salary variable so that it can be used in our constraint
salary = df["Salary"]

# since we need a constraint on the Position type, let us allocate the indexes belonging to positions FB,RB,WR,TE into a variable
# This wil help us retrieve the players for a particular position using the index list.
fb=df_fb.index.values.tolist()
rb=df_rb.index.values.tolist()
wr=df_wr.index.values.tolist()
te=df_te.index.values.tolist()

# declare decision variables which is the Xi that we had mentioned, thsi will tell us if a player is selected or not.
model.playerselected = Var(players, domain=Binary)   

#the objective is to maximize the points of the players selected
# The value of p in players will start from 1, hence we need to subtract 1 from it so that it refers to the actual index value in the dataframe
obj_expr = 0
for p in players:
  obj_expr += points[p-1]*model.playerselected[p]

# declare objective
#Model.team will give us the output of the total points earned by this team that is selected
model.team = Objective(
                      expr = obj_expr, # values come from the table
                      sense = maximize)

# declare constraints 
model.constraints = ConstraintList()
# There is a constriant on salary, position, mainly
# Salary constraint states that the sum of the salary must not exceed $50k
salary_player = 0
for p in players:
  salary_player+=salary[p-1]*model.playerselected[p]
model.constraints.add(salary_player <= 50000)

#Every fantasy football team has a certain number of players from each of the position
#The team must contain 9 players 
#the following cosntriants states that FB=2,RB=2,WR=2,TE=2
# following the above constraint to ensure that the team we build is strong and has the balance of skills to play the agme
# Position constraint on FB
# by running the loop, we are basically retrieving the index value of players with a position type and adding the corresponding player for that index+1 
# This will create a list with the players at that position (players are the binary variables)
# the final list must be equal to  2 or 3 based on the requirement

player_count_fb=0
for f in fb:
  player_count_fb += model.playerselected[f+1]
model.constraints.add(player_count_fb == 2)


# Position constraint on RB
player_count_rb=0
for r in rb:
  player_count_rb += model.playerselected[r+1]
model.constraints.add(player_count_rb == 2)


# Position constraint on WR
player_count_wr=0
for w in wr:
  player_count_wr += model.playerselected[w+1]
model.constraints.add(player_count_wr == 3)



# Position constraint on te
player_count_te=0
for t in te:
  player_count_te += model.playerselected[t+1]
model.constraints.add(player_count_te == 2)

# show the model you've created
model.pprint()


2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}
    playerselected_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :  385 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155

In [7]:
# solve it using the following package
SolverFactory('cbc', executable='/usr/bin/cbc').solve(model).write()
# show the results
print("The total points earned by the team is: ", model.team())

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 199.2
  Upper bound: 199.2
  Number of objectives: 1
  Number of constraints: 5
  Number of variables: 371
  Number of binary variables: 385
  Number of integer variables: 385
  Number of nonzeros: 229
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.03
  Wallclock time: 0.04
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bo

In [8]:
print("The total points earned by the team is: ", model.team())

The total points earned by the team is:  199.20000000000002


In [9]:
#now that we built the model, and we have the list of players and their statuses
# In order to print the details of the player list, we need to filter only the players that are selected
# create a list for the index
ind= list()
# this loop will retrieve all player variables that  has a value of 1 (it is a binary variable), these are the players that were selected 
# the value is stored as "p-1" as the player variable lsit starts from 1 in the model declaration while the player index starts at 0 in the dataframe
for p in players:
  if(model.playerselected[p]()==1):
     ind.append(p-1)
  else:
    continue
print(ind)


[0, 1, 4, 5, 7, 17, 23, 154, 190]


In [10]:
# The dataframe of the players that are selected for the team are as follows
df.iloc[ind]

,Position,Player,Week,Team,Opp,Opp Rank,Opp Position Rank,Salary,Points
0,WR,Deebo Samuel,10,SF,LAR,8,7700,7400,30.3
1,WR,Stefon Diggs,10,BUF,NYJ,8,7900,7500,33.2
4,RB,Rhamondre Stevenson,10,NE,CLE,10,5400,4500,30.4
5,RB,Darrel Williams,10,KC,LV,22,6000,5400,32.4
7,WR,Kendrick Bourne,10,NE,CLE,18,5500,4600,24.1
17,TE,Hunter Henry,10,NE,CLE,13,5400,4100,19.7
23,TE,Travis Kelce,10,KC,LV,28,7500,6900,22.9
154,FB,C.J. Ham,10,MIN,LAC,30,4500,4000,4.3
190,FB,Derek Watt,10,PIT,DET,31,4500,4000,1.9


In [11]:
# to see the list of player variables and their status in the team, 0 indicated they are not selected and 1 represents they are included
for p in players:
  print("player",p,":",model.playerselected[p]())

player 1 : 1.0
player 2 : 1.0
player 3 : 0.0
player 4 : 0.0
player 5 : 1.0
player 6 : 1.0
player 7 : 0.0
player 8 : 1.0
player 9 : 0.0
player 10 : 0.0
player 11 : 0.0
player 12 : 0.0
player 13 : 0.0
player 14 : 0.0
player 15 : 0.0
player 16 : 0.0
player 17 : 0.0
player 18 : 1.0
player 19 : 0.0
player 20 : 0.0
player 21 : 0.0
player 22 : 0.0
player 23 : 0.0
player 24 : 1.0
player 25 : 0.0
player 26 : 0.0
player 27 : 0.0
player 28 : 0.0
player 29 : 0.0
player 30 : 0.0
player 31 : 0.0
player 32 : 0.0
player 33 : 0.0
player 34 : 0.0
player 35 : 0.0
player 36 : 0.0
player 37 : 0.0
player 38 : 0.0
player 39 : 0.0
player 40 : 0.0
player 41 : 0.0
player 42 : 0.0
player 43 : 0.0
player 44 : 0.0
player 45 : 0.0
player 46 : 0.0
player 47 : 0.0
player 48 : 0.0
player 49 : 0.0
player 50 : 0.0
player 51 : 0.0
player 52 : 0.0
player 53 : 0.0
player 54 : 0.0
player 55 : 0.0
player 56 : 0.0
player 57 : 0.0
player 58 : 0.0
player 59 : 0.0
player 60 : 0.0
player 61 : 0.0
player 62 : 0.0
player 63 : 0.0
p